In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
df_Label = pd.read_csv('/home/exp-1/Desktop/data/ds3_labels.csv', sep=r'\s*,\s*', header=0, encoding='cp1252', engine='python')
df_Label.dropna()

,Dst Port,Protocol,Timestamp,Label,date,id
0,22,6,22/02/2018 08:26:03,Benign,2018-02-22 08:26:03,2018-02-22 08:26:036
1,34989,6,22/02/2018 08:26:24,Benign,2018-02-22 08:26:24,2018-02-22 08:26:246
2,500,17,22/02/2018 08:25:10,Benign,2018-02-22 08:25:10,2018-02-22 08:25:1017
3,500,17,22/02/2018 08:25:10,Benign,2018-02-22 08:25:10,2018-02-22 08:25:1017
4,500,17,22/02/2018 08:24:59,Benign,2018-02-22 08:24:59,2018-02-22 08:24:5917
...,...,...,...,...,...,...
2097145,54116,6,23/02/2018 09:00:54,Benign,2018-02-23 09:00:54,2018-02-23 09:00:546
2097146,33914,6,23/02/2018 12:31:42,Benign,2018-02-23 12:31:42,2018-02-23 12:31:426
2097147,53,17,23/02/2018 12:15:24,Benign,2018-02-23 12:15:24,2018-02-23 12:15:2417
2097148,3389,6,23/02/2018 12:45:39,Benign,2018-02-23 12:45:39,2018-02-23 12:45:396


In [3]:
print(df_Label)

         Dst Port  Protocol            Timestamp   Label                 date  \
0              22         6  22/02/2018 08:26:03  Benign  2018-02-22 08:26:03   
1           34989         6  22/02/2018 08:26:24  Benign  2018-02-22 08:26:24   
2             500        17  22/02/2018 08:25:10  Benign  2018-02-22 08:25:10   
3             500        17  22/02/2018 08:25:10  Benign  2018-02-22 08:25:10   
4             500        17  22/02/2018 08:24:59  Benign  2018-02-22 08:24:59   
...           ...       ...                  ...     ...                  ...   
2097145     54116         6  23/02/2018 09:00:54  Benign  2018-02-23 09:00:54   
2097146     33914         6  23/02/2018 12:31:42  Benign  2018-02-23 12:31:42   
2097147        53        17  23/02/2018 12:15:24  Benign  2018-02-23 12:15:24   
2097148      3389         6  23/02/2018 12:45:39  Benign  2018-02-23 12:45:39   
2097149       445         6  23/02/2018 09:04:11  Benign  2018-02-23 09:04:11   

                           

In [6]:
df_Label['id'].value_counts()

2018-02-22 08:28:006     385
2018-02-22 08:28:336     372
2018-02-23 10:18:006     334
2018-02-22 10:25:506     331
2018-02-22 08:32:146     313
                        ... 
2018-02-23 12:35:580       1
2018-02-22 08:52:200       1
2018-02-23 04:32:050       1
2018-02-23 12:30:200       1
2018-02-23 04:07:2617      1
Name: id, Length: 132535, dtype: int64

In [7]:
df_Label['Label'].value_counts()

Benign              2096222
Brute Force -Web        611
Brute Force -XSS        230
SQL Injection            87
Name: Label, dtype: int64

In [49]:
import json
Alerts = []
for line in open('/home/exp-1/Desktop/data/Alerts/Snort_dataset_3/snort_18.json', 'r'):
    Alerts.append(json.loads(line))

In [50]:
df_alerts = pd.json_normalize(Alerts)

In [51]:
col_list = ['event.source-ip','event.destination-ip','event.sport-itype','event.dport-icode','event.protocol','event.event-second','event.event-microsecond']
df_alerts = df_alerts[col_list].dropna()

In [52]:
df_alerts['date'] = pd.to_datetime(df_alerts['event.event-second'], unit='s').dropna()

In [53]:
df_alerts['id_count']= df_alerts['date'].astype(str)+df_alerts['event.protocol'].astype(int).astype(str)

In [54]:
df_alerts['id']=df_alerts['event.source-ip']+'-'+df_alerts['event.destination-ip']+'-'+df_alerts['event.sport-itype'].astype(int).astype(str)+'-'+df_alerts['event.dport-icode'].astype(int).astype(str)

In [61]:
df_alerts['id'].value_counts()

78.158.196.78-172.31.66.42-57982-445       87
222.186.46.26-172.31.66.62-1365-135        17
222.186.46.26-172.31.64.67-3628-135        11
180.248.12.14-172.31.67.50-58837-445       10
103.218.3.30-172.31.67.119-3612-135         8
                                           ..
1.171.88.240-172.31.65.81-55558-445         1
213.202.230.37-172.31.67.10-2780-3389       1
213.202.230.206-172.31.66.41-21005-3389     1
213.202.230.143-172.31.67.80-35954-3389     1
213.202.230.217-172.31.65.29-37853-3389     1
Name: id, Length: 282797, dtype: int64

In [56]:
#df_alerts = df_alerts.drop_duplicates(subset = 'id_count', keep = 'first')

In [57]:
df_alerts['POSITIVES'] = (df_alerts['id_count'].isin(df_Label['id'])) & (df_Label['Label']!='Benign')

In [58]:
df_alerts['POSITIVES'].value_counts()

False    286825
True        362
Name: POSITIVES, dtype: int64

In [59]:
df_Label_1 = df_Label.drop(df_alerts['id_count'].isin(df_Label['id']).index)

In [60]:
df_Label_1['Label'].value_counts()

Benign              1809397
Brute Force -Web        362
Brute Force -XSS        151
SQL Injection            53
Name: Label, dtype: int64

In [3]:
import pandas as pd

In [18]:
#df_alerts.to_csv('/home/exp-1/Desktop/data/alerts_processed/Snort_19_1_labels.csv')

In [7]:
df_s=pd.read_csv('/home/exp-1/Desktop/data/Alerts/Alerts_csv/ds2_snort/2019.csv')

In [8]:
print(df_s)

      event.source-ip event.destination-ip  event.sport-itype  \
0        192.168.10.3        192.168.10.19               88.0   
1        192.168.10.3        192.168.10.19               88.0   
2        192.168.10.3        192.168.10.19               88.0   
3        192.168.10.3        192.168.10.19               88.0   
4        192.168.10.3        192.168.10.19               88.0   
...               ...                  ...                ...   
29904   68.67.178.196        192.168.10.14               80.0   
29905   192.168.10.14        74.119.118.82            51633.0   
29906   192.168.10.15        38.72.130.155            61245.0   
29907   74.119.118.74        192.168.10.51               80.0   
29908   68.67.178.110         192.168.10.8               80.0   

       event.dport-icode  event.protocol  event.event-second  \
0                41567.0             6.0        1.499342e+09   
1                41567.0             6.0        1.499342e+09   
2                41569.0   

In [9]:
df_s['event.msg'].value_counts()

stream5: Reset outside window                                                10937
http_inspect: UNKNOWN METHOD                                                  7983
stream5: TCP Small Segment Threshold Exceeded                                 4434
http_inspect: NO CONTENT-LENGTH OR TRANSFER-ENCODING IN HTTP RESPONSE         2852
http_inspect: LONG HEADER                                                     1402
sensitive_data: sensitive data global threshold exceeded                      1091
stream5: Bad segment, overlap adjusted size less than/equal 0                  503
sensitive_data: sensitive data - eMail addresses                               154
stream5: TCP Timestamp is missing                                              151
http_inspect: OVERSIZE REQUEST-URI DIRECTORY                                   102
http_inspect: MESSAGE WITH INVALID CONTENT-LENGTH OR CHUNK SIZE                 95
stream5: Data sent on stream not accepting data                                 84
stre